# RAG with Minsearch

In [26]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

--2025-06-10 15:04:54--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5820 (5.7K) [text/plain]
Saving to: ‘minsearch.py.5’

minsearch.py.5      100%[===================>]   5.68K  --.-KB/s    in 0s      

2025-06-10 15:04:54 (28.5 MB/s) - ‘minsearch.py.5’ saved [5820/5820]



In [27]:
import minsearch

In [28]:
import json

In [29]:
with open ('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [30]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [31]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [32]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [33]:
q = 'the course has already started, can I still enroll?'

In [34]:
index.fit(documents)

In [35]:
boost = {'question' : 3.0, 'section' : 0.5}
results = index.search(
    query = q,
    filter_dict = {'course' : 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results  = 5
)



In [36]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [37]:
import os
from openai import OpenAI

api_key = os.getenv("OPENROUTER_API_KEY")

client = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)

response = client.chat.completions.create(
    model="deepseek/deepseek-r1-0528-qwen3-8b:free",
    messages=[{"role": "user", "content": q}]
)

print(response.choices[0].message.content)


Yes, it's **often possible**, though it depends entirely on the specific **course, institution, and educator**.

Here's what you should consider and do:

1.  **Check the official enrollment procedures:** Your school or platform might have specific rules for late enrollment, especially after the course has started.
2.  **Contact the instructor(s):** This is the **most important step**. Reaching out directly to the instructor(s) teaching the course is usually the best way to determine your options.
3.  **Contact the department or registrar:** Ask the appropriate administrative contact (usually the course department office or the main registrar) if spaces are still available for new students (though this might depend on waitlists, not necessarily late enrollment).
4.  **Be clear, concise, and polite:** When contacting the instructor and/or administrator, explain the situation briefly and politely.
    *   Example email to instructor: *Hi Professor X, I saw that [Course Name]'s enrollment 

In [38]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer ,output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [39]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [40]:
prompt = prompt_template.format(question = q, context = context).strip()

In [41]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer ,output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.


In [42]:
response = client.chat.completions.create(
    model="deepseek/deepseek-r1-0528-qwen3-8b:free",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

Yes, even if you don't register before the start, you're still eligible to join after the course has started, as long as you submit your homeworks before the deadlines.


#  RAG with Elasticsearch

In [62]:
def search(query):
    boost = {'question' : 5.0, 'section' : 0.5}
    
    results = index.search(
    query = query,
    filter_dict = {'course' : 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results  = 10
    )

    return results


In [44]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.


QUESTION: {question}

CONTEXT:
{context}
""".strip()
    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [45]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer ,output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.


In [46]:
def llm(prompt):
    response = client.chat.completions.create(
    model="deepseek/deepseek-r1-0528-qwen3-8b:free",
    messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


In [47]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [48]:
query = 'how do I run kafka ?'
rag(query)

'In the project directory, run:  \njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java'

**Search with Elasticsearch**

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
es_client.info()

In [55]:
index_name = "course-questions"

# Check if the index already exists
if not es_client.indices.exists(index=index_name):
    print(f"Index '{index_name}' does not exist. Creating it now...")
    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"}
            }
        }
    }
    es_client.indices.create(index=index_name, body=index_settings)
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists. Skipping creation.")

Index 'course-questions' already exists. Skipping creation.


In [56]:
from tqdm.auto import tqdm 

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [57]:
for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 327.87it/s]


In [58]:
elastic_query = 'can i join the course,i just found it?'

In [59]:
def elastic_search(elastic_query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": elastic_query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    } 

    response = es_client.search(index = index_name, body = search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [60]:
def rag(elastic_query):
    search_results = elastic_search(elastic_query)
    prompt = build_prompt(elastic_query, search_results)
    answer = llm(prompt)
    return answer

In [61]:
rag(elastic_query)

'According to the course information:\n\nYes, you can join the course even after the start date, and you are still eligible to submit the homeworks.'